In [42]:
import csv
from tkinter import filedialog

import matplotlib.pyplot as plt


# #bit clunky in a notebook
# stdFilePath = "data"
# selectedFile = filedialog.askopenfilename(initialdir=stdFilePath,
#                                           title="Select file",
#                                           filetypes=(("Only logs", "*.csv"),
#                                                      ("all files", "*.*")))
# if selectedFile == "":
#     exit("[ERR] Please select a file")


selectedFile = "data/example.csv"


ages = []

with open(selectedFile, "r") as f:
    reader = csv.DictReader(f, delimiter=',')
    # next(reader, None)  # skip a line
    KeyToDesc = reader.__next__()
    FullNameToKey = {v: k for k, v in KeyToDesc.items()}  # maybe usefull

    # reader = csv.DictReader(f, fieldnames=names, delimiter=',')

    for row in reader:
        # print(row)
        ages.append(row["DD01_01"])
        print("Case:", row["CASE"], row["STARTED"], "Alter:",
              row["DD01_01"], KeyToDesc["SS01_01"] + ":", row["SS01_01"])


Case: 2 2023-04-25 17:09:10 Alter: 2 AR FeelingScale: Unzufrieden: 2
Case: 3 2023-04-25 17:25:10 Alter:  AR FeelingScale: Unzufrieden: -9
Case: 4 2023-04-25 17:25:31 Alter:  AR FeelingScale: Unzufrieden: -9


In [ ]:
plt.plot(ages)
plt.ylabel("y")
plt.xlabel("x")
plt.show()